### First experiments with U-Net on real world slp data

Try U-Net following [Xiantao et al., 2020](https://www.nature.com/articles/s41598-020-59801-x), but with only 4 convolutions, opposed to 5.
Work with real world slp samples. Omit final row, to have number of rows evenly divideable by 2 for all three max pooling operations.

Choose **mask type**: Can either have random sparsity mask, individually for each data sample ('variable'), or create only a single random mask, that is then applied to all samples identically ('fixed'). 
And optionally extend data with **augmentation factor**: Specify the number of times, each sample is to be cloned, keeping the original order.

#### Experiments:

1) **mask_type='fixed', augmentation_factor=1:** Base experiment, according to szenario to have limitet number of measurements from stations, that are fixed in their location. Fast to train, low ressources required.
2) **mask_type='variable', augmentation_factor=1:** Extends base experiment, can handle variable inputs with fixed sparsity in single model. Fits to observations from e.g. argo floats on their trajectory or well suited for e.g. sea surface temperature measurements, where varying cloud coverage limits infrared observations. In its simples form, use each sample only once. Expect worse performance, commpared to base experiment.
3) **mask_type='variable', augmentation_factor>0:** Improve performance by using each input sample multiple times, allowed by random sparsity. Try to reduce overfitting and increase generalization on unseen data. Comes at the expense of increased training time.
4) **mask_type='optimal', augmentation_factor=1:** Use optimal mask from relevance experiment with range model.

Use **sparse and scaled (to [0,1]) samples as inputs**. Use **complete and scaled (to [0,1]) samples as targets**.

Got results on nesh, using GPU cluster node with batch **unet_4conv_slp_realworld.py**. Specify **mask_type** and **augmentation_factor**!

#### Results:

In general, get some nice results: Model predictions capture the main structure in slp anomalies, even for *sparsity=0.99*.

In terms of mean squarred error on training and validation data, models from our base experiment (experiment 1) trained on **fixed** sparsity mask **outperformes** models trained on **variable** mask, at least **without data augmentation** (experiment 2), as expected. But it's a trade off, since models from experiment 2 have higher flexibility: Can feed samples with specified sparsity but can freely choose, *which* inputs to use. The price is - as mentioned - a slightly worse reconstruction quality, which can be hardly recognized with human eye.

Adding data augmentation **factor=2** (experiment 3), model performance significantly improves and beats performance from our base experiment, keeping full flexibility in a sense, that we can freely choose input pixels, at least if we keep sparsity fixed.

**Note:** Need multi-run experiments for each setup, show error bars. And run experiment 3 with higher augmentation factors to validate the story.

In [1]:
import os
import sys
sys.path.append('../reconstruct_missing_data')

from pathlib import Path
from json import dump, load

import numpy as np
from matplotlib import pyplot as plt

from data_loading import find_data_files, load_data_set, get_anomalies, clone_data, create_missing_mask, split_and_scale_data
from models import build_unet_4conv

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda, concatenate, Conv1D, Conv2D, MaxPool2D, UpSampling2D, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow.keras.initializers as tfi
import tensorflow.keras.regularizers as tfr
from tensorflow.keras.utils import plot_model

# Suppress Tensorflow warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
# Set working directory, according to working directory in scripts:
os.chdir('/gxfs_work1/geomar/smomw511')

In [3]:
## Specify base parameters for experiments to compare:

# exp1: fixed mask
# exp2: variable mask, augmentation factor 1
# exp3a: variable mask, augmentation factor 2
# exp3b: variable mask, augmentation factor 3
# exp4: optimal mask

# Common parameters for all experiments:
model_config = 'unet_4conv'
feature = 'sea-level-pressure' # Choose either 'sea-level-pressure' or 'sea-surface-temperature' as feature.
feature_short = 'slp' # Free to set short name, to store results, e.g. 'slp' and 'sst'.
source = 'realworld' # Choose Earth System Model, either 'FOCI' or 'CESM', or 'realworld'.

# Specific parameters for exp1:
mask_types = ['fixed','fixed','fixed']
missing_types = ['discrete','discrete','discrete']
augmentation_factors = [1,1,1]
runs = ['_seed_1', '_seed_2', '_seed_3']

# Reconstruct path to experiment:
paths = []
for i in range(len(mask_types)):
    paths.append(Path('GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/'+model_config+'_'+feature_short+'_'+source+'_'
                      +mask_types[i]+'_'+missing_types[i]+'_factor_'+str(augmentation_factors[i])+runs[i]))
    print(paths[i])

# Initialize storage for parameters of all experiments:
parameters_all = []

# Reload parameters for all experiments:
for path in paths:
    
    # Open corresponding parameter file and store:
    with open(path / 'parameters.json', 'r') as f:
        parameters_all.append(load(f))

# Convert to numpy array:
parameters_all = np.array(parameters_all)

# Extract certain parameters from first experiment. Note: These parameters should be identical for all experiments of interest, here.
train_val_split = parameters_all[0]['train_val_split']
missing_values = parameters_all[0]['missing_values']
scale_to = parameters_all[0]['scale_to']
epochs = parameters_all[0]['epochs']

## Get an overview of final train and validation loss, depending on rel. amount of missing values, for all experiments.
## Note: Start with epoch 0, hence untrained model.

# Initialize storage for final train and validation loss. Dimension: (#experiments, #missing values settings, #epochs+1)
train_loss_final_all = np.zeros((len(paths),len(missing_values),epochs+1))
val_loss_final_all = np.zeros((len(paths),len(missing_values),epochs+1))

# Loop over experiments:
for i in range(len(paths)):
    
    # Get path for current experiment:
    path = paths[i]

    # Loop over array of desired rel. amounts of missing values:
    for j in range(len(missing_values)):

        # Get current sparsity:
        missing = missing_values[j]

        # Reload loss.
        # Rel. amount of missing values = 0.999 requires special treatment:
        if missing==0.999:
            train_loss = np.load(path / 'missing_' f'{int(missing*1000)}'/ 'train_loss.npy')
            val_loss = np.load(path / 'missing_' f'{int(missing*1000)}'/ 'val_loss.npy')
        else:
            train_loss = np.load(path / 'missing_' f'{int(missing*100)}'/ 'train_loss.npy')
            val_loss = np.load(path / 'missing_' f'{int(missing*100)}'/ 'val_loss.npy')
            
        # Store final train and validation loss:
        train_loss_final_all[i,j,:] = train_loss
        val_loss_final_all[i,j,:] = val_loss
        
# Get minimum train and validation loss over epochs for experiment 1:
train_loss_min_exp1 = np.min(train_loss_final_all,axis=-1)
val_loss_min_exp1 = np.min(val_loss_final_all,axis=-1)

# Get mean and std dev of minimum train and validation loss over all runs:
train_loss_min_mean_exp1 = np.mean(train_loss_min_exp1,axis=0)
val_loss_min_mean_exp1 = np.mean(val_loss_min_exp1,axis=0)
train_loss_min_std_exp1 = np.std(train_loss_min_exp1,axis=0)
val_loss_min_std_exp1 = np.std(val_loss_min_exp1,axis=0)

GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/unet_4conv_slp_realworld_fixed_discrete_factor_1_seed_1
GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/unet_4conv_slp_realworld_fixed_discrete_factor_1_seed_2
GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/unet_4conv_slp_realworld_fixed_discrete_factor_1_seed_3


In [4]:
### Find epch and run with minimum train loss:

## First step: Find for each run the epoch with minimum train loss and the according loss values.

# Initialize storage, dimensions: (#runs, #missing value settings)
train_loss_min_epoch = np.zeros((train_loss_final_all.shape[0],train_loss_final_all.shape[1]))
train_loss_min_value = np.zeros((train_loss_final_all.shape[0],train_loss_final_all.shape[1]))

# Loop over runs:
for i in range(train_loss_final_all.shape[0]):
    
    # Loop over missing value settings:
    for j in range (train_loss_final_all.shape[1]):
        
        # Find epoch with minimum loss:
        min_epoch = np.argsort(train_loss_final_all[i,j,:])[0]
        
        # Store epoch and according loss:
        train_loss_min_epoch[i,j] = min_epoch
        train_loss_min_value[i,j] = train_loss_final_all[i,j,min_epoch]
        
## Second step: Find the run with absolute minimum train loss.

# Initialize storage, dimensions: (#missing value settings)
train_loss_min_run = np.zeros(train_loss_final_all.shape[1])

# Loop over missing value settings:
for j in range(train_loss_final_all.shape[1]):
    
    # Find run with minimum loss:
    min_run = np.argsort(train_loss_min_value[:,j])[0]
    
    # Store run number:
    train_loss_min_run[j] = min_run

## Output for each missing value setting the run / epoch combination with best performance (lowest train loss):

# Loop over missing value settings:
for j in range(train_loss_final_all.shape[1]):
    
    # Rel. amount of missing values = 0.999 requires special treatment:
    if missing_values[j]==0.999:
        
        # Output: Copy optimal models
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*1000)}'+
            '/epoch_'+str(int(train_loss_min_epoch[int(train_loss_min_run[j]),j]))+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*1000)}'+'/model'
        )

        # Output: Copy missing masks
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*1000)}'+
            '/missing_mask.npy'+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*1000)}'
        )
        
    else:
        
        # Output: Copy optimal models
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*100)}'+
            '/epoch_'+str(int(train_loss_min_epoch[int(train_loss_min_run[j]),j]))+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*100)}'+'/model'
        )

        # Output: Copy missing masks
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*100)}'+
            '/missing_mask.npy'+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*100)}'
        )
    
print("\n")

cp -r unet_4conv_slp_realworld_fixed_discrete_factor_1_seed_2/missing_999/epoch_10 unet_4conv_slp_realworld_fixed_discrete_factor_1_final/missing_999/model
cp -r unet_4conv_slp_realworld_fixed_discrete_factor_1_seed_2/missing_999/missing_mask.npy unet_4conv_slp_realworld_fixed_discrete_factor_1_final/missing_999
cp -r unet_4conv_slp_realworld_fixed_discrete_factor_1_seed_3/missing_99/epoch_10 unet_4conv_slp_realworld_fixed_discrete_factor_1_final/missing_99/model
cp -r unet_4conv_slp_realworld_fixed_discrete_factor_1_seed_3/missing_99/missing_mask.npy unet_4conv_slp_realworld_fixed_discrete_factor_1_final/missing_99
cp -r unet_4conv_slp_realworld_fixed_discrete_factor_1_seed_2/missing_95/epoch_9 unet_4conv_slp_realworld_fixed_discrete_factor_1_final/missing_95/model
cp -r unet_4conv_slp_realworld_fixed_discrete_factor_1_seed_2/missing_95/missing_mask.npy unet_4conv_slp_realworld_fixed_discrete_factor_1_final/missing_95
cp -r unet_4conv_slp_realworld_fixed_discrete_factor_1_seed_3/missi

In [5]:
# Specific parameters for exp2:
mask_types = ['variable','variable','variable']
missing_types = ['discrete','discrete','discrete']
augmentation_factors = [1,1,1]
runs = ['_seed_1', '_seed_2', '_seed_3']

# Reconstruct path to experiment:
paths = []
for i in range(len(mask_types)):
    paths.append(Path('GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/'+model_config+'_'+feature_short+'_'+source+'_'
                      +mask_types[i]+'_'+missing_types[i]+'_factor_'+str(augmentation_factors[i])+runs[i]))
    print(paths[i])

# Initialize storage for parameters of all experiments:
parameters_all = []

# Reload parameters for all experiments:
for path in paths:
    
    # Open corresponding parameter file and store:
    with open(path / 'parameters.json', 'r') as f:
        parameters_all.append(load(f))

# Convert to numpy array:
parameters_all = np.array(parameters_all)

# Extract certain parameters from first experiment. Note: These parameters should be identical for all experiments of interest, here.
train_val_split = parameters_all[0]['train_val_split']
missing_values = parameters_all[0]['missing_values']
scale_to = parameters_all[0]['scale_to']
epochs = parameters_all[0]['epochs']

## Get an overview of final train and validation loss, depending on rel. amount of missing values, for all experiments.
## Note: Start with epoch 0, hence untrained model.

# Initialize storage for final train and validation loss. Dimension: (#experiments, #missing values settings, #epochs+1)
train_loss_final_all = np.zeros((len(paths),len(missing_values),epochs+1))
val_loss_final_all = np.zeros((len(paths),len(missing_values),epochs+1))

# Loop over experiments:
for i in range(len(paths)):
    
    # Get path for current experiment:
    path = paths[i]

    # Loop over array of desired rel. amounts of missing values:
    for j in range(len(missing_values)):

        # Get current sparsity:
        missing = missing_values[j]

        # Reload loss.
        # Rel. amount of missing values = 0.999 requires special treatment:
        if missing==0.999:
            train_loss = np.load(path / 'missing_' f'{int(missing*1000)}'/ 'train_loss.npy')
            val_loss = np.load(path / 'missing_' f'{int(missing*1000)}'/ 'val_loss.npy')
        else:
            train_loss = np.load(path / 'missing_' f'{int(missing*100)}'/ 'train_loss.npy')
            val_loss = np.load(path / 'missing_' f'{int(missing*100)}'/ 'val_loss.npy')
 
        # Store final train and validation loss:
        train_loss_final_all[i,j,:] = train_loss
        val_loss_final_all[i,j,:] = val_loss
        
# Get minimum train and validation loss over epochs for experiment 2:
train_loss_min_exp2 = np.min(train_loss_final_all,axis=-1)
val_loss_min_exp2 = np.min(val_loss_final_all,axis=-1)

# Get mean and std dev of minimum train and validation loss over all runs:
train_loss_min_mean_exp2 = np.mean(train_loss_min_exp2,axis=0)
val_loss_min_mean_exp2 = np.mean(val_loss_min_exp2,axis=0)
train_loss_min_std_exp2 = np.std(train_loss_min_exp2,axis=0)
val_loss_min_std_exp2 = np.std(val_loss_min_exp2,axis=0)

GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/unet_4conv_slp_realworld_variable_discrete_factor_1_seed_1
GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/unet_4conv_slp_realworld_variable_discrete_factor_1_seed_2
GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/unet_4conv_slp_realworld_variable_discrete_factor_1_seed_3


In [6]:
### Find epoch and run with minimum train loss:

## First step: Find for each run the epoch with minimum train loss and the according loss values.

# Initialize storage, dimensions: (#runs, #missing value settings)
train_loss_min_epoch = np.zeros((train_loss_final_all.shape[0],train_loss_final_all.shape[1]))
train_loss_min_value = np.zeros((train_loss_final_all.shape[0],train_loss_final_all.shape[1]))

# Loop over runs:
for i in range(train_loss_final_all.shape[0]):
    
    # Loop over missing value settings:
    for j in range (train_loss_final_all.shape[1]):
        
        # Find epoch with minimum loss:
        min_epoch = np.argsort(train_loss_final_all[i,j,:])[0]
        
        # Store epoch and according loss:
        train_loss_min_epoch[i,j] = min_epoch
        train_loss_min_value[i,j] = train_loss_final_all[i,j,min_epoch]
        
## Second step: Find the run with absolute minimum train loss.

# Initialize storage, dimensions: (#missing value settings)
train_loss_min_run = np.zeros(train_loss_final_all.shape[1])

# Loop over missing value settings:
for j in range(train_loss_final_all.shape[1]):
    
    # Find run with minimum loss:
    min_run = np.argsort(train_loss_min_value[:,j])[0]
    
    # Store run number:
    train_loss_min_run[j] = min_run

## Output for each missing value setting the run / epoch combination with best performance (lowest train loss):

# Loop over missing value settings:
for j in range(train_loss_final_all.shape[1]):
    
    # Rel. amount of missing values = 0.999 requires special treatment:
    if missing_values[j]==0.999:
        
        # Output: Copy optimal models
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*1000)}'+
            '/epoch_'+str(int(train_loss_min_epoch[int(train_loss_min_run[j]),j]))+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*1000)}'+'/model'
        )

        # Output: Copy missing masks
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*1000)}'+
            '/missing_mask.npy'+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*1000)}'
        )
        
    else:
        
        # Output: Copy optimal models
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*100)}'+
            '/epoch_'+str(int(train_loss_min_epoch[int(train_loss_min_run[j]),j]))+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*100)}'+'/model'
        )

        # Output: Copy missing masks
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*100)}'+
            '/missing_mask.npy'+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*100)}'
        )

print("\n")

cp -r unet_4conv_slp_realworld_variable_discrete_factor_1_seed_1/missing_999/epoch_10 unet_4conv_slp_realworld_variable_discrete_factor_1_final/missing_999/model
cp -r unet_4conv_slp_realworld_variable_discrete_factor_1_seed_1/missing_999/missing_mask.npy unet_4conv_slp_realworld_variable_discrete_factor_1_final/missing_999
cp -r unet_4conv_slp_realworld_variable_discrete_factor_1_seed_3/missing_99/epoch_10 unet_4conv_slp_realworld_variable_discrete_factor_1_final/missing_99/model
cp -r unet_4conv_slp_realworld_variable_discrete_factor_1_seed_3/missing_99/missing_mask.npy unet_4conv_slp_realworld_variable_discrete_factor_1_final/missing_99
cp -r unet_4conv_slp_realworld_variable_discrete_factor_1_seed_3/missing_95/epoch_10 unet_4conv_slp_realworld_variable_discrete_factor_1_final/missing_95/model
cp -r unet_4conv_slp_realworld_variable_discrete_factor_1_seed_3/missing_95/missing_mask.npy unet_4conv_slp_realworld_variable_discrete_factor_1_final/missing_95
cp -r unet_4conv_slp_realworld

In [7]:
# Specific parameters for exp3a:
mask_types = ['variable','variable','variable']
missing_types = ['discrete','discrete','discrete']
augmentation_factors = [2,2,2]
runs = ['_seed_1', '_seed_2', '_seed_3']

# Reconstruct path to experiment:
paths = []
for i in range(len(mask_types)):
    paths.append(Path('GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/'+model_config+'_'+feature_short+'_'+source+'_'
                      +mask_types[i]+'_'+missing_types[i]+'_factor_'+str(augmentation_factors[i])+runs[i]))
    print(paths[i])

# Initialize storage for parameters of all experiments:
parameters_all = []

# Reload parameters for all experiments:
for path in paths:
    
    # Open corresponding parameter file and store:
    with open(path / 'parameters.json', 'r') as f:
        parameters_all.append(load(f))

# Convert to numpy array:
parameters_all = np.array(parameters_all)

# Extract certain parameters from first experiment. Note: These parameters should be identical for all experiments of interest, here.
train_val_split = parameters_all[0]['train_val_split']
missing_values = parameters_all[0]['missing_values']
scale_to = parameters_all[0]['scale_to']
epochs = parameters_all[0]['epochs']

## Get an overview of final train and validation loss, depending on rel. amount of missing values, for all experiments.
## Note: Start with epoch 0, hence untrained model.

# Initialize storage for final train and validation loss. Dimension: (#experiments, #missing values settings, #epochs+1)
train_loss_final_all = np.zeros((len(paths),len(missing_values),epochs+1))
val_loss_final_all = np.zeros((len(paths),len(missing_values),epochs+1))

# Loop over experiments:
for i in range(len(paths)):
    
    # Get path for current experiment:
    path = paths[i]

    # Loop over array of desired rel. amounts of missing values:
    for j in range(len(missing_values)):

        # Get current sparsity:
        missing = missing_values[j]

        # Reload loss.
        # Rel. amount of missing values = 0.999 requires special treatment:
        if missing==0.999:
            train_loss = np.load(path / 'missing_' f'{int(missing*1000)}'/ 'train_loss.npy')
            val_loss = np.load(path / 'missing_' f'{int(missing*1000)}'/ 'val_loss.npy')
        else:
            train_loss = np.load(path / 'missing_' f'{int(missing*100)}'/ 'train_loss.npy')
            val_loss = np.load(path / 'missing_' f'{int(missing*100)}'/ 'val_loss.npy')
 
        # Store final train and validation loss:
        train_loss_final_all[i,j,:] = train_loss
        val_loss_final_all[i,j,:] = val_loss
        
# Get minimum train and validation loss over epochs for experiment 3:
train_loss_min_exp3a = np.min(train_loss_final_all,axis=-1)
val_loss_min_exp3a = np.min(val_loss_final_all,axis=-1)

# Get mean and std dev of minimum train and validation loss over all runs:
train_loss_min_mean_exp3a = np.mean(train_loss_min_exp3a,axis=0)
val_loss_min_mean_exp3a = np.mean(val_loss_min_exp3a,axis=0)
train_loss_min_std_exp3a = np.std(train_loss_min_exp3a,axis=0)
val_loss_min_std_exp3a = np.std(val_loss_min_exp3a,axis=0)

GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/unet_4conv_slp_realworld_variable_discrete_factor_2_seed_1
GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/unet_4conv_slp_realworld_variable_discrete_factor_2_seed_2
GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/unet_4conv_slp_realworld_variable_discrete_factor_2_seed_3


In [8]:
### Find epoch and run with minimum train loss:

## First step: Find for each run the epoch with minimum train loss and the according loss values.

# Initialize storage, dimensions: (#runs, #missing value settings)
train_loss_min_epoch = np.zeros((train_loss_final_all.shape[0],train_loss_final_all.shape[1]))
train_loss_min_value = np.zeros((train_loss_final_all.shape[0],train_loss_final_all.shape[1]))

# Loop over runs:
for i in range(train_loss_final_all.shape[0]):
    
    # Loop over missing value settings:
    for j in range (train_loss_final_all.shape[1]):
        
        # Find epoch with minimum loss:
        min_epoch = np.argsort(train_loss_final_all[i,j,:])[0]
        
        # Store epoch and according loss:
        train_loss_min_epoch[i,j] = min_epoch
        train_loss_min_value[i,j] = train_loss_final_all[i,j,min_epoch]
        
## Second step: Find the run with absolute minimum train loss.

# Initialize storage, dimensions: (#missing value settings)
train_loss_min_run = np.zeros(train_loss_final_all.shape[1])

# Loop over missing value settings:
for j in range(train_loss_final_all.shape[1]):
    
    # Find run with minimum loss:
    min_run = np.argsort(train_loss_min_value[:,j])[0]
    
    # Store run number:
    train_loss_min_run[j] = min_run

## Output for each missing value setting the run / epoch combination with best performance (lowest train loss):

# Loop over missing value settings:
for j in range(train_loss_final_all.shape[1]):
    
    # Rel. amount of missing values = 0.999 requires special treatment:
    if missing_values[j]==0.999:
        
        # Output: Copy optimal models
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*1000)}'+
            '/epoch_'+str(int(train_loss_min_epoch[int(train_loss_min_run[j]),j]))+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*1000)}'+'/model'
        )

        # Output: Copy missing masks
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*1000)}'+
            '/missing_mask.npy'+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*1000)}'
        )
        
    else:
        
        # Output: Copy optimal models
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*100)}'+
            '/epoch_'+str(int(train_loss_min_epoch[int(train_loss_min_run[j]),j]))+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*100)}'+'/model'
        )

        # Output: Copy missing masks
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*100)}'+
            '/missing_mask.npy'+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*100)}'
        )
        
print("\n")

cp -r unet_4conv_slp_realworld_variable_discrete_factor_2_seed_2/missing_999/epoch_9 unet_4conv_slp_realworld_variable_discrete_factor_2_final/missing_999/model
cp -r unet_4conv_slp_realworld_variable_discrete_factor_2_seed_2/missing_999/missing_mask.npy unet_4conv_slp_realworld_variable_discrete_factor_2_final/missing_999
cp -r unet_4conv_slp_realworld_variable_discrete_factor_2_seed_3/missing_99/epoch_10 unet_4conv_slp_realworld_variable_discrete_factor_2_final/missing_99/model
cp -r unet_4conv_slp_realworld_variable_discrete_factor_2_seed_3/missing_99/missing_mask.npy unet_4conv_slp_realworld_variable_discrete_factor_2_final/missing_99
cp -r unet_4conv_slp_realworld_variable_discrete_factor_2_seed_2/missing_95/epoch_10 unet_4conv_slp_realworld_variable_discrete_factor_2_final/missing_95/model
cp -r unet_4conv_slp_realworld_variable_discrete_factor_2_seed_2/missing_95/missing_mask.npy unet_4conv_slp_realworld_variable_discrete_factor_2_final/missing_95
cp -r unet_4conv_slp_realworld_

In [9]:
# Specific parameters for exp3b:
mask_types = ['variable','variable','variable']
missing_types = ['discrete','discrete','discrete']
augmentation_factors = [3,3,3]
runs = ['_seed_1','_seed_2','_seed_3']

# Reconstruct path to experiment:
paths = []
for i in range(len(mask_types)):
    paths.append(Path('GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/'+model_config+'_'+feature_short+'_'+source+'_'
                      +mask_types[i]+'_'+missing_types[i]+'_factor_'+str(augmentation_factors[i])+runs[i]))
    print(paths[i])

# Initialize storage for parameters of all experiments:
parameters_all = []

# Reload parameters for all experiments:
for path in paths:
    
    # Open corresponding parameter file and store:
    with open(path / 'parameters.json', 'r') as f:
        parameters_all.append(load(f))

# Convert to numpy array:
parameters_all = np.array(parameters_all)

# Extract certain parameters from first experiment. Note: These parameters should be identical for all experiments of interest, here.
train_val_split = parameters_all[0]['train_val_split']
missing_values = parameters_all[0]['missing_values']
scale_to = parameters_all[0]['scale_to']
epochs = parameters_all[0]['epochs']

## Get an overview of final train and validation loss, depending on rel. amount of missing values, for all experiments.
## Note: Start with epoch 0, hence untrained model.

# Initialize storage for final train and validation loss. Dimension: (#experiments, #missing values settings, #epochs+1)
train_loss_final_all = np.zeros((len(paths),len(missing_values),epochs+1))
val_loss_final_all = np.zeros((len(paths),len(missing_values),epochs+1))

# Loop over experiments:
for i in range(len(paths)):
    
    # Get path for current experiment:
    path = paths[i]

    # Loop over array of desired rel. amounts of missing values:
    for j in range(len(missing_values)):

        # Get current sparsity:
        missing = missing_values[j]

        # Reload loss.
        # Rel. amount of missing values = 0.999 requires special treatment:
        if missing==0.999:
            train_loss = np.load(path / 'missing_' f'{int(missing*1000)}'/ 'train_loss.npy')
            val_loss = np.load(path / 'missing_' f'{int(missing*1000)}'/ 'val_loss.npy')
        else:
            train_loss = np.load(path / 'missing_' f'{int(missing*100)}'/ 'train_loss.npy')
            val_loss = np.load(path / 'missing_' f'{int(missing*100)}'/ 'val_loss.npy')
 
        # Store final train and validation loss:
        train_loss_final_all[i,j,:] = train_loss
        val_loss_final_all[i,j,:] = val_loss
        
# Get minimum train and validation loss over epochs for experiment 3:
train_loss_min_exp3b = np.min(train_loss_final_all,axis=-1)
val_loss_min_exp3b = np.min(val_loss_final_all,axis=-1)

# Get mean and std dev of minimum train and validation loss over all runs:
train_loss_min_mean_exp3b = np.mean(train_loss_min_exp3b,axis=0)
val_loss_min_mean_exp3b = np.mean(val_loss_min_exp3b,axis=0)
train_loss_min_std_exp3b = np.std(train_loss_min_exp3b,axis=0)
val_loss_min_std_exp3b = np.std(val_loss_min_exp3b,axis=0)

GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/unet_4conv_slp_realworld_variable_discrete_factor_3_seed_1
GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/unet_4conv_slp_realworld_variable_discrete_factor_3_seed_2
GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/unet_4conv_slp_realworld_variable_discrete_factor_3_seed_3


In [10]:
### Find epch and run with minimum train loss:

## First step: Find for each run the epoch with minimum train loss and the according loss values.

# Initialize storage, dimensions: (#runs, #missing value settings)
train_loss_min_epoch = np.zeros((train_loss_final_all.shape[0],train_loss_final_all.shape[1]))
train_loss_min_value = np.zeros((train_loss_final_all.shape[0],train_loss_final_all.shape[1]))

# Loop over runs:
for i in range(train_loss_final_all.shape[0]):
    
    # Loop over missing value settings:
    for j in range (train_loss_final_all.shape[1]):
        
        # Find epoch with minimum loss:
        min_epoch = np.argsort(train_loss_final_all[i,j,:])[0]
        
        # Store epoch and according loss:
        train_loss_min_epoch[i,j] = min_epoch
        train_loss_min_value[i,j] = train_loss_final_all[i,j,min_epoch]
        
## Second step: Find the run with absolute minimum train loss.

# Initialize storage, dimensions: (#missing value settings)
train_loss_min_run = np.zeros(train_loss_final_all.shape[1])

# Loop over missing value settings:
for j in range(train_loss_final_all.shape[1]):
    
    # Find run with minimum loss:
    min_run = np.argsort(train_loss_min_value[:,j])[0]
    
    # Store run number:
    train_loss_min_run[j] = min_run

## Output for each missing value setting the run / epoch combination with best performance (lowest train loss):

# Loop over missing value settings:
for j in range(train_loss_final_all.shape[1]):
    
    # Rel. amount of missing values = 0.999 requires special treatment:
    if missing_values[j]==0.999:
        
        # Output: Copy optimal models
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*1000)}'+
            '/epoch_'+str(int(train_loss_min_epoch[int(train_loss_min_run[j]),j]))+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*1000)}'+'/model'
        )

        # Output: Copy missing masks
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*1000)}'+
            '/missing_mask.npy'+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*1000)}'
        )
        
    else:
        
        # Output: Copy optimal models
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*100)}'+
            '/epoch_'+str(int(train_loss_min_epoch[int(train_loss_min_run[j]),j]))+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*100)}'+'/model'
        )

        # Output: Copy missing masks
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_seed_'+str(int(train_loss_min_run[j])+1)+'/missing_' f'{int(missing_values[j]*100)}'+
            '/missing_mask.npy'+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*100)}'
        )
print("\n")

cp -r unet_4conv_slp_realworld_variable_discrete_factor_3_seed_2/missing_999/epoch_9 unet_4conv_slp_realworld_variable_discrete_factor_3_final/missing_999/model
cp -r unet_4conv_slp_realworld_variable_discrete_factor_3_seed_2/missing_999/missing_mask.npy unet_4conv_slp_realworld_variable_discrete_factor_3_final/missing_999
cp -r unet_4conv_slp_realworld_variable_discrete_factor_3_seed_1/missing_99/epoch_10 unet_4conv_slp_realworld_variable_discrete_factor_3_final/missing_99/model
cp -r unet_4conv_slp_realworld_variable_discrete_factor_3_seed_1/missing_99/missing_mask.npy unet_4conv_slp_realworld_variable_discrete_factor_3_final/missing_99
cp -r unet_4conv_slp_realworld_variable_discrete_factor_3_seed_2/missing_95/epoch_10 unet_4conv_slp_realworld_variable_discrete_factor_3_final/missing_95/model
cp -r unet_4conv_slp_realworld_variable_discrete_factor_3_seed_2/missing_95/missing_mask.npy unet_4conv_slp_realworld_variable_discrete_factor_3_final/missing_95
cp -r unet_4conv_slp_realworld_

In [15]:
# Specific parameters for exp4a:
mask_types = ['optimal_from_CESM']
missing_types = ['discrete']
augmentation_factors = [1]
runs = ['_final']

# Reconstruct path to experiment:
paths = []
for i in range(len(mask_types)):
    paths.append(Path('GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/'+model_config+'_'+feature_short+'_'+source+'_'
                      +mask_types[i]+'_'+missing_types[i]+'_factor_'+str(augmentation_factors[i])+runs[i]))
    print(paths[i])

# Initialize storage for parameters of all experiments:
parameters_all = []

# Reload parameters for all experiments:
for path in paths:
    
    # Open corresponding parameter file and store:
    with open(path / 'parameters.json', 'r') as f:
        parameters_all.append(load(f))

# Convert to numpy array:
parameters_all = np.array(parameters_all)

# Extract certain parameters from first experiment. Note: These parameters should be identical for all experiments of interest, here.
train_val_split = parameters_all[0]['train_val_split']
missing_values_exp4 = parameters_all[0]['missing_values']
scale_to = parameters_all[0]['scale_to']
epochs = parameters_all[0]['epochs']

## Get an overview of final train and validation loss, depending on rel. amount of missing values, for all experiments.
## Note: Start with epoch 0, hence untrained model.

# Initialize storage for final train and validation loss. Dimension: (#experiments, #missing values settings, #epochs+1)
train_loss_final_all = np.zeros((len(paths),len(missing_values_exp4),epochs+1))
val_loss_final_all = np.zeros((len(paths),len(missing_values_exp4),epochs+1))

# Loop over experiments:
for i in range(len(paths)):
    
    # Get path for current experiment:
    path = paths[i]

    # Loop over array of desired rel. amounts of missing values:
    for j in range(len(missing_values_exp4)):

        # Get current sparsity:
        missing = missing_values_exp4[j]

        # Reload loss.
        # Rel. amount of missing values = 0.999 requires special treatment:
        if missing==0.999:
            train_loss = np.load(path / 'missing_' f'{int(missing*1000)}'/ 'train_loss.npy')
            val_loss = np.load(path / 'missing_' f'{int(missing*1000)}'/ 'val_loss.npy')
        else:
            train_loss = np.load(path / 'missing_' f'{int(missing*100)}'/ 'train_loss.npy')
            val_loss = np.load(path / 'missing_' f'{int(missing*100)}'/ 'val_loss.npy')
            
        # Store final train and validation loss:
        train_loss_final_all[i,j,:] = train_loss
        val_loss_final_all[i,j,:] = val_loss
        
# Get minimum train and validation loss over epochs for experiment 1:
train_loss_min_exp4a = np.min(train_loss_final_all,axis=-1)
val_loss_min_exp4a = np.min(val_loss_final_all,axis=-1)

# Get mean and std dev of minimum train and validation loss over all runs:
train_loss_min_mean_exp4a = np.mean(train_loss_min_exp4a,axis=0)
val_loss_min_mean_exp4a = np.mean(val_loss_min_exp4a,axis=0)
train_loss_min_std_exp4a = np.std(train_loss_min_exp4a,axis=0)
val_loss_min_std_exp4a = np.std(val_loss_min_exp4a,axis=0)

GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/unet_4conv_slp_realworld_optimal_from_CESM_discrete_factor_1_final


In [16]:
### Find epch and run with minimum train loss:

## First step: Find for each run the epoch with minimum train loss and the according loss values.

# Initialize storage, dimensions: (#runs, #missing value settings)
train_loss_min_epoch = np.zeros((train_loss_final_all.shape[0],train_loss_final_all.shape[1]))
train_loss_min_value = np.zeros((train_loss_final_all.shape[0],train_loss_final_all.shape[1]))

# Loop over runs:
for i in range(train_loss_final_all.shape[0]):
    
    # Loop over missing value settings:
    for j in range (train_loss_final_all.shape[1]):
        
        # Find epoch with minimum loss:
        min_epoch = np.argsort(train_loss_final_all[i,j,:])[0]
        
        # Store epoch and according loss:
        train_loss_min_epoch[i,j] = min_epoch
        train_loss_min_value[i,j] = train_loss_final_all[i,j,min_epoch]
        
## Second step: Find the run with absolute minimum train loss.

# Initialize storage, dimensions: (#missing value settings)
train_loss_min_run = np.zeros(train_loss_final_all.shape[1])

# Loop over missing value settings:
for j in range(train_loss_final_all.shape[1]):
    
    # Find run with minimum loss:
    min_run = np.argsort(train_loss_min_value[:,j])[0]
    
    # Store run number:
    train_loss_min_run[j] = min_run

## Output for each missing value setting the run / epoch combination with best performance (lowest train loss):

# Loop over missing value settings:
for j in range(train_loss_final_all.shape[1]):
    
    # Rel. amount of missing values = 0.999 requires special treatment:
    if missing_values[j]==0.999:
        
        # Output: Copy optimal models
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_run_2'+'/missing_' f'{int(missing_values[j]*1000)}'+
            '/epoch_'+str(int(train_loss_min_epoch[int(train_loss_min_run[j]),j]))+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*1000)}'+'/model'
        )

        # Output: Copy missing masks
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_run_2'+'/missing_' f'{int(missing_values[j]*1000)}'+
            '/missing_mask.npy'+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*1000)}'
        )
        
    else:
        
        # Output: Copy optimal models
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_run_2'+'/missing_' f'{int(missing_values[j]*100)}'+
            '/epoch_'+str(int(train_loss_min_epoch[int(train_loss_min_run[j]),j]))+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*100)}'+'/model'
        )

        # Output: Copy missing masks
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_run_2'+'/missing_' f'{int(missing_values[j]*100)}'+
            '/missing_mask.npy'+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*100)}'
        )
print("\n")

cp -r unet_4conv_slp_realworld_optimal_from_CESM_discrete_factor_1_run_2/missing_999/epoch_9 unet_4conv_slp_realworld_optimal_from_CESM_discrete_factor_1_final/missing_999/model
cp -r unet_4conv_slp_realworld_optimal_from_CESM_discrete_factor_1_run_2/missing_999/missing_mask.npy unet_4conv_slp_realworld_optimal_from_CESM_discrete_factor_1_final/missing_999
cp -r unet_4conv_slp_realworld_optimal_from_CESM_discrete_factor_1_run_2/missing_99/epoch_10 unet_4conv_slp_realworld_optimal_from_CESM_discrete_factor_1_final/missing_99/model
cp -r unet_4conv_slp_realworld_optimal_from_CESM_discrete_factor_1_run_2/missing_99/missing_mask.npy unet_4conv_slp_realworld_optimal_from_CESM_discrete_factor_1_final/missing_99
cp -r unet_4conv_slp_realworld_optimal_from_CESM_discrete_factor_1_run_2/missing_95/epoch_8 unet_4conv_slp_realworld_optimal_from_CESM_discrete_factor_1_final/missing_95/model
cp -r unet_4conv_slp_realworld_optimal_from_CESM_discrete_factor_1_run_2/missing_95/missing_mask.npy unet_4co

In [17]:
# Specific parameters for exp4b:
mask_types = ['optimal_from_FOCI']
missing_types = ['discrete']
augmentation_factors = [1]
runs = ['_final']

# Reconstruct path to experiment:
paths = []
for i in range(len(mask_types)):
    paths.append(Path('GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/'+model_config+'_'+feature_short+'_'+source+'_'
                      +mask_types[i]+'_'+missing_types[i]+'_factor_'+str(augmentation_factors[i])+runs[i]))
    print(paths[i])

# Initialize storage for parameters of all experiments:
parameters_all = []

# Reload parameters for all experiments:
for path in paths:
    
    # Open corresponding parameter file and store:
    with open(path / 'parameters.json', 'r') as f:
        parameters_all.append(load(f))

# Convert to numpy array:
parameters_all = np.array(parameters_all)

# Extract certain parameters from first experiment. Note: These parameters should be identical for all experiments of interest, here.
train_val_split = parameters_all[0]['train_val_split']
missing_values_exp4 = parameters_all[0]['missing_values']
scale_to = parameters_all[0]['scale_to']
epochs = parameters_all[0]['epochs']

## Get an overview of final train and validation loss, depending on rel. amount of missing values, for all experiments.
## Note: Start with epoch 0, hence untrained model.

# Initialize storage for final train and validation loss. Dimension: (#experiments, #missing values settings, #epochs+1)
train_loss_final_all = np.zeros((len(paths),len(missing_values_exp4),epochs+1))
val_loss_final_all = np.zeros((len(paths),len(missing_values_exp4),epochs+1))

# Loop over experiments:
for i in range(len(paths)):
    
    # Get path for current experiment:
    path = paths[i]

    # Loop over array of desired rel. amounts of missing values:
    for j in range(len(missing_values_exp4)):

        # Get current sparsity:
        missing = missing_values_exp4[j]

        # Reload loss.
        # Rel. amount of missing values = 0.999 requires special treatment:
        if missing==0.999:
            train_loss = np.load(path / 'missing_' f'{int(missing*1000)}'/ 'train_loss.npy')
            val_loss = np.load(path / 'missing_' f'{int(missing*1000)}'/ 'val_loss.npy')
        else:
            train_loss = np.load(path / 'missing_' f'{int(missing*100)}'/ 'train_loss.npy')
            val_loss = np.load(path / 'missing_' f'{int(missing*100)}'/ 'val_loss.npy')
            
        # Store final train and validation loss:
        train_loss_final_all[i,j,:] = train_loss
        val_loss_final_all[i,j,:] = val_loss
        
# Get minimum train and validation loss over epochs for experiment 1:
train_loss_min_exp4b = np.min(train_loss_final_all,axis=-1)
val_loss_min_exp4b = np.min(val_loss_final_all,axis=-1)

# Get mean and std dev of minimum train and validation loss over all runs:
train_loss_min_mean_exp4b = np.mean(train_loss_min_exp4b,axis=0)
val_loss_min_mean_exp4b = np.mean(val_loss_min_exp4b,axis=0)
train_loss_min_std_exp4b = np.std(train_loss_min_exp4b,axis=0)
val_loss_min_std_exp4b = np.std(val_loss_min_exp4b,axis=0)

GitGeomar/marco-landt-hayen/reconstruct_missing_data_results/unet_4conv_slp_realworld_optimal_from_FOCI_discrete_factor_1_final


In [18]:
### Find epch and run with minimum train loss:

## First step: Find for each run the epoch with minimum train loss and the according loss values.

# Initialize storage, dimensions: (#runs, #missing value settings)
train_loss_min_epoch = np.zeros((train_loss_final_all.shape[0],train_loss_final_all.shape[1]))
train_loss_min_value = np.zeros((train_loss_final_all.shape[0],train_loss_final_all.shape[1]))

# Loop over runs:
for i in range(train_loss_final_all.shape[0]):
    
    # Loop over missing value settings:
    for j in range (train_loss_final_all.shape[1]):
        
        # Find epoch with minimum loss:
        min_epoch = np.argsort(train_loss_final_all[i,j,:])[0]
        
        # Store epoch and according loss:
        train_loss_min_epoch[i,j] = min_epoch
        train_loss_min_value[i,j] = train_loss_final_all[i,j,min_epoch]
        
## Second step: Find the run with absolute minimum train loss.

# Initialize storage, dimensions: (#missing value settings)
train_loss_min_run = np.zeros(train_loss_final_all.shape[1])

# Loop over missing value settings:
for j in range(train_loss_final_all.shape[1]):
    
    # Find run with minimum loss:
    min_run = np.argsort(train_loss_min_value[:,j])[0]
    
    # Store run number:
    train_loss_min_run[j] = min_run

## Output for each missing value setting the run / epoch combination with best performance (lowest train loss):

# Loop over missing value settings:
for j in range(train_loss_final_all.shape[1]):
    
    # Rel. amount of missing values = 0.999 requires special treatment:
    if missing_values[j]==0.999:
        
        # Output: Copy optimal models
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_run_2'+'/missing_' f'{int(missing_values[j]*1000)}'+
            '/epoch_'+str(int(train_loss_min_epoch[int(train_loss_min_run[j]),j]))+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*1000)}'+'/model'
        )

        # Output: Copy missing masks
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_run_2'+'/missing_' f'{int(missing_values[j]*1000)}'+
            '/missing_mask.npy'+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*1000)}'
        )
        
    else:
        
        # Output: Copy optimal models
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_run_2'+'/missing_' f'{int(missing_values[j]*100)}'+
            '/epoch_'+str(int(train_loss_min_epoch[int(train_loss_min_run[j]),j]))+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*100)}'+'/model'
        )

        # Output: Copy missing masks
        print(
            'cp -r '+model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_run_2'+'/missing_' f'{int(missing_values[j]*100)}'+
            '/missing_mask.npy'+' '+
            model_config+'_'+feature_short+'_'+source+'_'+mask_types[0]+'_'+missing_types[0]+'_factor_'+
            str(augmentation_factors[0])+'_final'+'/missing_' f'{int(missing_values[j]*100)}'
        )
print("\n")

cp -r unet_4conv_slp_realworld_optimal_from_FOCI_discrete_factor_1_run_2/missing_999/epoch_9 unet_4conv_slp_realworld_optimal_from_FOCI_discrete_factor_1_final/missing_999/model
cp -r unet_4conv_slp_realworld_optimal_from_FOCI_discrete_factor_1_run_2/missing_999/missing_mask.npy unet_4conv_slp_realworld_optimal_from_FOCI_discrete_factor_1_final/missing_999
cp -r unet_4conv_slp_realworld_optimal_from_FOCI_discrete_factor_1_run_2/missing_99/epoch_10 unet_4conv_slp_realworld_optimal_from_FOCI_discrete_factor_1_final/missing_99/model
cp -r unet_4conv_slp_realworld_optimal_from_FOCI_discrete_factor_1_run_2/missing_99/missing_mask.npy unet_4conv_slp_realworld_optimal_from_FOCI_discrete_factor_1_final/missing_99
cp -r unet_4conv_slp_realworld_optimal_from_FOCI_discrete_factor_1_run_2/missing_95/epoch_9 unet_4conv_slp_realworld_optimal_from_FOCI_discrete_factor_1_final/missing_95/model
cp -r unet_4conv_slp_realworld_optimal_from_FOCI_discrete_factor_1_run_2/missing_95/missing_mask.npy unet_4co

In [19]:
# Set colors for individual experiments:
colors = ['black', 'blue', 'red', 'cyan', 'magenta', 'yellow']
linestyles = ['--', '--', '--', '--', '--', '--']

# Plot final train and validation loss over sparsity for all experiments:
fig, axes = plt.subplots(1,2,figsize=(14,4))

# exp1:
axes[0].plot(
        np.log10(train_loss_min_mean_exp1), color=colors[0],
        label = 'mask: fixed',
        marker = 'o', linestyle = linestyles[0],
    )

axes[1].plot(
        np.log10(val_loss_min_mean_exp1), color=colors[0],
        label = 'mask: fixed',
        marker = 'o', linestyle = linestyles[0],
    )

# exp2:
axes[0].plot(
        np.log10(train_loss_min_mean_exp2), color=colors[1],
        label = 'mask: variable / discrete, augmentation factor: 1',
        marker = 'o', linestyle = linestyles[1],
    )

axes[1].plot(
        np.log10(val_loss_min_mean_exp2), color=colors[1],
        label = 'mask: variable / discrete, augmentation factor: 1',
        marker = 'o', linestyle = linestyles[1],
    )

# exp3a:
axes[0].plot(
        np.log10(train_loss_min_mean_exp3a), color=colors[2],
        label = 'mask: variable / discrete, augmentation factor: 2',
        marker = 'o', linestyle = linestyles[2],
    )

axes[1].plot(
        np.log10(val_loss_min_mean_exp3a), color=colors[2],
        label = 'mask: variable / discrete, augmentation factor: 2',
        marker = 'o', linestyle = linestyles[2],
    )

# exp3b:
axes[0].plot(
        np.log10(train_loss_min_mean_exp3b), color=colors[3],
        label = 'mask: variable / discrete, augmentation factor: 3',
        marker = 'o', linestyle = linestyles[3],
    )

axes[1].plot(
        np.log10(val_loss_min_mean_exp3b), color=colors[3],
        label = 'mask: variable / discrete, augmentation factor: 3',
        marker = 'o', linestyle = linestyles[3],
    )

# exp4a:
axes[0].plot(
        np.log10(train_loss_min_mean_exp4a), color=colors[4],
        label = 'mask: optimal from CESM',
        marker = 'o', linestyle = linestyles[4],
    )

axes[1].plot(
        np.log10(val_loss_min_mean_exp4a), color=colors[4],
        label = 'mask: optimal from CESM',
        marker = 'o', linestyle = linestyles[4],
    )

# exp4b:
axes[0].plot(
        np.log10(train_loss_min_mean_exp4b), color=colors[5],
        label = 'mask: optimal from FOCI',
        marker = 'o', linestyle = linestyles[5],
    )

axes[1].plot(
        np.log10(val_loss_min_mean_exp4b), color=colors[5],
        label = 'mask: optimal from FOCI',
        marker = 'o', linestyle = linestyles[5],
    )

# #plt.ylim(0.6, 0.8)
axes[0].set_xticks(np.arange(0, len(missing_values), step=1), missing_values, fontsize=12)
axes[1].set_xticks(np.arange(0, len(missing_values), step=1), missing_values, fontsize=12)
axes[0].set_title('SLP real world: Mean training loss', fontsize=16)
axes[1].set_title('SLP real world: Mean validation loss', fontsize=16)
axes[0].set_xlabel('rate of missing values', fontsize=12)
axes[1].set_xlabel('rate of missing values', fontsize=12)
axes[0].set_ylabel('log. mean squared error', fontsize=12)
axes[0].legend()
axes[1].legend()
plt.show() 